In [1]:
from peft import prepare_model_for_kbit_training,LoraConfig,get_peft_model
from transformers import AutoModelForCausalLM,AutoTokenizer,TrainingArguments,Trainer,default_data_collator
from datasets import load_dataset
import evaluate
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

[2025-06-13 21:38:17,661] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)


W0613 21:38:18.872000 24104 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


[2025-06-13 21:38:19,287] [INFO] [logging.py:107:log_dist] [Rank -1] [TorchCheckpointEngine] Initialized with serialization = False


In [2]:
model_name="Qwen/Qwen3-0.6B"
model=AutoModelForCausalLM.from_pretrained(model_name,device_map='auto',torch_dtype="auto",)
tokenizer=AutoTokenizer.from_pretrained(model_name)

In [3]:
model

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [4]:
model = prepare_model_for_kbit_training(model)

In [5]:
lora_config = LoraConfig(
    r=16,  # LoRA矩阵的秩
    lora_alpha=32,  # LoRA alpha参数
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # 要应用LoRA的模块
    lora_dropout=0.05,  # Dropout概率
    bias="none",  # 是否训练偏置
    task_type="CAUSAL_LM",  # 任务类型
)

In [6]:
def print_module_names(model, prefix=""):
    for name, module in model.named_children():
        full_name = f"{prefix}.{name}" if prefix else name
        print(full_name)
        print_module_names(module, full_name)

print_module_names(model)

model
model.embed_tokens
model.layers
model.layers.0
model.layers.0.self_attn
model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.o_proj
model.layers.0.self_attn.q_norm
model.layers.0.self_attn.k_norm
model.layers.0.mlp
model.layers.0.mlp.gate_proj
model.layers.0.mlp.up_proj
model.layers.0.mlp.down_proj
model.layers.0.mlp.act_fn
model.layers.0.input_layernorm
model.layers.0.post_attention_layernorm
model.layers.1
model.layers.1.self_attn
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.o_proj
model.layers.1.self_attn.q_norm
model.layers.1.self_attn.k_norm
model.layers.1.mlp
model.layers.1.mlp.gate_proj
model.layers.1.mlp.up_proj
model.layers.1.mlp.down_proj
model.layers.1.mlp.act_fn
model.layers.1.input_layernorm
model.layers.1.post_attention_layernorm
model.layers.2
model.layers.2.self_attn
model.layers.2.self_attn.q_proj
model.layers.2.self_attn

In [7]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # 打印可训练参数

trainable params: 4,587,520 || all params: 600,637,440 || trainable%: 0.7638


In [8]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 1024)
        (layers): ModuleList(
          (0-27): 28 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1024, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

### 加载数据

In [9]:
dataset = load_dataset('json', data_files={
    'train': '../数据集/data/train.json',
    'validation': '../数据集/data/eval.json'
})

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['conversations'],
        num_rows: 320
    })
    validation: Dataset({
        features: ['conversations'],
        num_rows: 30
    })
})

### 数据预处理

In [11]:
def process_fun(example):
    question=[]
    answer=[]
    for i in example['conversations']:
        for j in i:
            if j['from']=='human':
                question.append(j['value'])
            elif j['from']=='gpt':
                answer.append(j['value'])
    return {'question':question,'answer':answer}

In [12]:
process_data=dataset.map(process_fun,batched=True,remove_columns=dataset['train'].column_names) # 提取问题和答案数据预处理

In [13]:
def tokenizer_fun(examples):
    # 构建完整的指令格式（问：{问题}\n答：{答案}）
    instructions = []
    for q, a in zip(examples['question'], examples['answer']):
        instruction = f"问：{q}\n答：{a}"
        instructions.append(instruction)
    
    encoded = tokenizer(
        instructions,
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    
    labels = encoded["input_ids"].clone()
    
    # 定位"答："的位置，标记需要预测的部分
    answer_start_token = tokenizer.encode("答：", add_special_tokens=False)[0]
    
    # 遍历批次中的每个样本
    for i in range(len(labels)):
        # 找到每个样本中"答："的第一个token位置
        answer_positions = (labels[i] == answer_start_token).nonzero(as_tuple=True)[0]
        if len(answer_positions) > 0:
            # 只取第一个"答："的位置
            first_answer_pos = answer_positions[0]
            # 将"答："之前的token标记为-100（忽略计算损失）
            labels[i, :first_answer_pos] = -100
    
    return {
        "input_ids": encoded["input_ids"],
        "attention_mask": encoded["attention_mask"],
        "labels": labels
    }

In [14]:
tokenized_dataset = process_data.map(
    tokenizer_fun,
    batched=True,
    remove_columns=process_data['train'].column_names
)

In [15]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 606
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 52
    })
})

### 训练超参数配置

In [16]:
# 定义训练参数
training_args = TrainingArguments(
    output_dir="./lora_train_qwen_0.6B_model",
    logging_steps=100,
    logging_dir='./runs',
    eval_strategy='epoch',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    gradient_accumulation_steps=4,  # 如果GPU内存有限
)

### 训练模型

In [18]:
trainer=Trainer(
    model=model,
    args=training_args,
    eval_dataset=tokenized_dataset["validation"],
    train_dataset=tokenized_dataset["train"],
    data_collator=default_data_collator,
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [19]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,4.549963
2,No log,2.204734
3,4.080700,1.684535


TrainOutput(global_step=114, training_loss=3.7788990422299036, metrics={'train_runtime': 14912.0498, 'train_samples_per_second': 0.122, 'train_steps_per_second': 0.008, 'total_flos': 2485585755242496.0, 'train_loss': 3.7788990422299036, 'epoch': 3.0})

In [20]:
trainer.save_model('./lora_train_qwen')